In [1]:
pip install langchain langchain-openai openai dateparser langfuse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.3/294.3 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 16.7 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

import os

# Replace with your actual key or keep as-is if it's already in a variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
import os
from typing import Callable, Dict, List, Tuple, Optional

from pydantic import BaseModel
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_functions_agent, AgentExecutor

from langfuse import Langfuse, get_client
from langfuse.langchain import CallbackHandler

In [4]:
# Initialize Langfuse tracing client at startup
Langfuse(
    public_key=os.environ.get("LANGFUSE_PUBLIC_KEY", "pk-lf-a5edd784-d1c7-4c6c-a1d3-6c8288285b65"),
    secret_key=os.environ.get("LANGFUSE_SECRET_KEY", "sk-lf-79ef80bf-d991-4e2e-9f58-9b468aaccccd"),
    host=os.environ.get("LANGFUSE_HOST", "https://us.cloud.langfuse.com"),
)
langfuse = get_client()
langfuse_handler = CallbackHandler()

In [5]:
# Global list to track tool usage during each interaction
TOOL_USAGE_LOG: List[str] = []

class ChatbotResponse(BaseModel):
    """
    Structured output of the chatbot for logging or downstream usage.
    """
    thought: str
    message: str
    tools: List[str]


def tracked_tool(name: str, func: Callable[[str], str]) -> Callable[[str], str]:
    """
    Wraps a tool function to log its name when called.

    Args:
        name: Tool identifier.
        func: Underlying processing function.

    Returns:
        Wrapper that logs usage and invokes the tool.
    """
    def wrapper(input_text: str) -> str:
        TOOL_USAGE_LOG.append(name)
        return func(input_text)
    return wrapper


def simple_calculator(query: str) -> str:
    """
    Evaluates a math expression.

    Args:
        query: Expression string.

    Returns:
        Computed result or error message.
    """
    try:
        result: float = eval(query, {"__builtins__": {}})
        return str(result)
    except Exception as err:
        return f"Error in calculation: {err}"


def get_calculator_tool() -> Tool:
    """
    Builds calculator tool with logging wrapper.

    Returns:
        Tool instance.
    """
    return Tool(
        name="Calculator",
        description="Evaluates basic math expressions.",
        func=tracked_tool("Calculator", simple_calculator),
    )


def get_llm() -> ChatOpenAI:
    """
    Initializes the OpenAI chat model.

    Returns:
        ChatOpenAI instance.

    Raises:
        ValueError: If the API key is not set.
    """
    api_key: Optional[str] = os.environ.get("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OPENAI_API_KEY environment variable is not set.")
    return ChatOpenAI(model="gpt-4", temperature=0.0, openai_api_key=api_key)


def get_prompt() -> ChatPromptTemplate:
    """
    Constructs and returns the prompt structure for the agent.
    """
    return ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that can use tools."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])


def get_agent_chain_and_tools() -> Tuple[AgentExecutor, List[Tool]]:
    """
    Constructs the AgentExecutor with Langfuse callback for tracing.

    Returns:
        Tuple of AgentExecutor and list of tool instances.
    """
    llm = get_llm()
    tools = [get_calculator_tool(),]
    prompt = get_prompt()

    # Create a tool-enabled agent
    agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)

    # Attach Langfuse tracing callback
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        callbacks=[langfuse_handler],
    )

    return agent_executor, tools


def run_chat_loop(agent_chain: AgentExecutor, tools: List[Tool]) -> None:
    """
    Runs the interactive CLI loop, tracing each message.

    Args:
        agent_chain: Configured AgentExecutor with tracing.
        tools: Available tool instances.
    """
    print("Chatbot is ready. Type 'exit' to quit.")
    while True:
        user_input: str = input("User: ")
        if user_input.lower() in {"exit", "quit"}:
            print("Exiting chatbot.")
            break

        try:
            TOOL_USAGE_LOG.clear()
            raw_response: Dict[str, str] = agent_chain.invoke({"input": user_input})
            used_tools = TOOL_USAGE_LOG.copy()

            thought = (
                f"Processed using tool(s): {', '.join(used_tools)}"
                if used_tools else "No tools used."
            )

            structured = ChatbotResponse(
                thought=thought,
                message=raw_response.get("output", "[No output]"),
                tools=used_tools,
            )
            print("Structured Response:", structured.model_dump_json(indent=2))
        except Exception as err:
            print("Error during chat:", err)

    # Flush trace data to Langfuse so it appears in the UI
    langfuse.flush()

In [6]:
agent_chain, tool_list = get_agent_chain_and_tools()
run_chat_loop(agent_chain, tool_list)

Chatbot is ready. Type 'exit' to quit.
User: tell me a joke


> Entering new AgentExecutor chain...


Sure, here's a classic one for you:

Why don't scientists trust atoms?

Because they make up everything!

> Finished chain.
Structured Response: {
  "thought": "No tools used.",
  "message": "Sure, here's a classic one for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!",
  "tools": []
}
User: what is 3+5-19/23


> Entering new AgentExecutor chain...

Invoking: `Calculator` with `3+5-19/23`


7.173913043478261

The result of the expression 3+5-19/23 is approximately 7.17.

> Finished chain.
Structured Response: {
  "thought": "Processed using tool(s): Calculator",
  "message": "The result of the expression 3+5-19/23 is approximately 7.17.",
  "tools": [
    "Calculator"
  ]
}
User: what is 4-5*(9-1/18)


> Entering new AgentExecutor chain...

Invoking: `Calculator` with `4-5*(9-1/18)`


-40.72222222222222

The result of the expression 4-5*(9-1/18) is approximately -40.72.

> Finished chain.
Structured Response: {
  "thought": "Processed using tool(s): Calculator",
  "message": "The result of the expression 4-5*(9-1/18) is approximately -40.72.",
  "tools": [
    "Calculator"
  ]
}
User: quit
Exiting chatbot.
